[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vaishnavvk/Design2Style/blob/main/app.ipynb)
# Design2Style

Design2Style: Interactive Clothing Image Generation combines neural style transfer, U2-Net segmentation, and saliency mapping to transform and personalize clothing articles, revolutionizing fashion expression.

In [ ]:
#@title Mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Path
%cd drive/MyDrive/Mini_Project/app

/content/drive/MyDrive/Mini_Project/app


In [ ]:
#@title Edit configurations
import torch
import albumentations as A

from albumentations.pytorch import ToTensorV2

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

CHECKPOINT_DIR = './checkpoints'
U2NET_SALIENCY_MAP_CHECKPOINT_FILE = 'u2net.pth'
U2NET_CLOTHES_CHECKPOINT_FILE = 'cloth_segm_u2net_latest.pth'

ENABLE_STYLE_TRANSFER = True
IMG_SIZE = 356
ALPHA = 1
BETA = 0.01
EPOCH_CHECKPOINT = 100

basic_transform = A.Compose([
    A.Resize(height=IMG_SIZE, width=IMG_SIZE),
    A.Normalize(
        mean=(.5, .5, .5),
        std=(.5, .5, .5),
    ),
    ToTensorV2()
])

In [ ]:
#@title Utilities
import os
import numpy as np

from PIL import Image
from collections import OrderedDict


def open_image(path):
    return Image.open(path)


def load_image(img):
    img = open_image(img).convert()
    img = basic_transform(image=np.array(img))['image']
    img = img.unsqueeze(0)

    return img.to(DEVICE)


def load_essential_images(original_img, style_img):
    original_img = load_image(original_img)
    style_img = load_image(style_img)

    generated = original_img.clone().requires_grad_(True)

    return original_img, style_img, generated


def gram_matrix(feature_map):
    return feature_map.mm(feature_map.t())


def min_max_normalization(d):
    return (d - torch.min(d)) / (torch.max(d) - torch.min(d))


def checkpoint_exists(filename):
    return filename in os.listdir(CHECKPOINT_DIR)


def load_vgg_model():
    return VGG().to(DEVICE).eval()


def load_u2net_model(in_ch=3, out_ch=4, checkpoint=U2NET_CLOTHES_CHECKPOINT_FILE, ordered_dict=True):
    model = U2NET(in_ch, out_ch).to(DEVICE)

    if checkpoint_exists(checkpoint):
        checkpoint = torch.load(
            os.path.join(CHECKPOINT_DIR, checkpoint),
            map_location=DEVICE
        )

        if ordered_dict:
            state_dict = OrderedDict()

            for k, v in checkpoint.items():
                name = k[7:]
                state_dict[name] = v

            model.load_state_dict(state_dict)
        else:
            model.load_state_dict(checkpoint)

    model.eval()

    return model


In [ ]:
#@title Pipelines
import torch.optim as optim

from torchvision.utils import save_image


def to_tensor(array):
    return torch.tensor(np.array(array))


def style_transfer_pipeline(model, original_img, style_img, EPOCHS, LEARNING_RATE, output_image='generated.png'):
    original_img, style_img, generated = load_essential_images(original_img, style_img)

    optimizer = optim.Adam([generated], lr=LEARNING_RATE)

    for epoch in range(int(EPOCHS)):
        generated_features = model(generated)
        original_img_features = model(original_img)
        style_features = model(style_img)

        style_loss = original_loss = 0

        for gen_feature, orig_feature, style_feature in zip(generated_features, original_img_features, style_features):
            batch_size, channel, height, width = gen_feature.shape

            original_loss += torch.mean((gen_feature - orig_feature) ** 2)

            G = gram_matrix(gen_feature.view(channel, height * width))
            A = gram_matrix(style_feature.view(channel, height * width))

            style_loss += torch.mean((G - A) ** 2)

        total_loss = ALPHA * original_loss + BETA * style_loss

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        print(f'Epoch[{epoch}] Total loss = {total_loss}')

        if epoch % EPOCH_CHECKPOINT == 0:
            save_image(generated, output_image)


def segmentation_pipeline(model, original_image, output_image=None):
    original_image = basic_transform(image=np.array(open_image(original_image).convert('RGB')))['image']

    with torch.no_grad():
        p_image = original_image.to(DEVICE).unsqueeze(0)

        masks = model(p_image)

        masks = torch.max(masks[0], dim=1, keepdim=True)[1]
        masks = torch.squeeze(masks, dim=0)
        masks = torch.squeeze(masks, dim=0)

    masks = masks.cpu().unsqueeze(0)
    masks = min_max_normalization(masks).squeeze()

    predict_np = masks.numpy() * 255

    masks = Image.fromarray(predict_np.astype('uint8')).convert('L')
    masks = masks.resize((IMG_SIZE, IMG_SIZE), resample=3)

    if output_image is not None:
        masks.save(output_image)

    return masks


def saliency_map_pipeline(model, original_image, output_image='saliency_map.jpg'):
    original_image = load_image(original_image)

    with torch.no_grad():
        original_image = original_image.to(DEVICE)
        d1, d2, d3, d4, d5, d6, d7 = model(original_image)

        pred = d1[:, 0, :, :]

    pred = min_max_normalization(pred.cpu()).squeeze()
    pred = pred.numpy() * 255

    map = Image.fromarray(pred.astype('uint8')).convert('L')
    map = map.resize((IMG_SIZE, IMG_SIZE))

    del d1, d2, d3, d4, d5, d6, d7

    if output_image is not None:
        map.save(output_image)

    return map


def merge_images_pipeline(mask, saliency_map, original_image, generated_image, output_image='output.png'):
    original_image = open_image(original_image).convert()
    original_size = original_image.size
    original_image = original_image.resize((IMG_SIZE, IMG_SIZE))

    generated_image = open_image(generated_image).convert()

    merged_image = original_image.copy()

    pix_mask = mask.load()
    pix_saliency = saliency_map.convert('L').load()
    pix_gen = generated_image.load()
    pix_seg_gen = merged_image.load()

    for x in range(original_image.size[0]):
        for y in range(original_image.size[1]):
            if pix_mask[x, y] == 255:
                if pix_saliency[x, y] == 255:
                    pix_seg_gen[x, y] = tuple(int(i) for i in pix_gen[x, y])
                else:
                    saliency_binary_val = (pix_saliency[x, y] / 255)
                    final = tuple((saliency_binary_val * pix_gen[x, y][i] + (1 - saliency_binary_val) * pix_seg_gen[x, y][i]) for i in range(3))

                    pix_seg_gen[x, y] = tuple(int(i) for i in final)

    merged_image.resize(original_size).save(output_image)
    return output_image


In [ ]:
#@title Models
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F


def _upsample_like(src, tar):
    src = F.interpolate(src, size=tar.shape[2:], mode='bilinear')

    return src


class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()

        self.chosen_features = [0, 5, 10, 19, 28]
        self.model = models.vgg19(weights='DEFAULT').features[:28]

    def forward(self, x):
        features = []

        for idx, layer in enumerate(self.model):
            x = layer(x)

            if idx in self.chosen_features:
                features.append(x)

        return features


class REBNCONV(nn.Module):
    def __init__(self, in_ch=3, out_ch=3, dirate=1):
        super(REBNCONV, self).__init__()

        self.conv_s1 = nn.Conv2d(
            in_ch,
            out_ch,
            kernel_size=3,
            padding=1 * dirate,
            dilation=1 * dirate
        )
        self.bn_s1 = nn.BatchNorm2d(out_ch)
        self.relu_s1 = nn.ReLU(inplace=True)

    def forward(self, x):
        hx = x
        out = self.relu_s1(self.bn_s1(self.conv_s1(hx)))

        return out


class RSU7(nn.Module):
    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU7, self).__init__()

        self.rebnconvin = REBNCONV(in_ch, out_ch, dirate=1)

        self.rebnconv1 = REBNCONV(out_ch, mid_ch, dirate=1)
        self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv2 = REBNCONV(mid_ch, mid_ch, dirate=1)
        self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv3 = REBNCONV(mid_ch, mid_ch, dirate=1)
        self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv4 = REBNCONV(mid_ch, mid_ch, dirate=1)
        self.pool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv5 = REBNCONV(mid_ch, mid_ch, dirate=1)
        self.pool5 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv6 = REBNCONV(mid_ch, mid_ch, dirate=1)

        self.rebnconv7 = REBNCONV(mid_ch, mid_ch, dirate=2)

        self.rebnconv6d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv5d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv4d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv3d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv2d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv1d = REBNCONV(mid_ch * 2, out_ch, dirate=1)

    def forward(self, x):
        hx = x
        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx = self.pool1(hx1)

        hx2 = self.rebnconv2(hx)
        hx = self.pool2(hx2)

        hx3 = self.rebnconv3(hx)
        hx = self.pool3(hx3)

        hx4 = self.rebnconv4(hx)
        hx = self.pool4(hx4)

        hx5 = self.rebnconv5(hx)
        hx = self.pool5(hx5)

        hx6 = self.rebnconv6(hx)

        hx7 = self.rebnconv7(hx6)

        hx6d = self.rebnconv6d(torch.cat((hx7, hx6), 1))
        hx6dup = _upsample_like(hx6d, hx5)

        hx5d = self.rebnconv5d(torch.cat((hx6dup, hx5), 1))
        hx5dup = _upsample_like(hx5d, hx4)

        hx4d = self.rebnconv4d(torch.cat((hx5dup, hx4), 1))
        hx4dup = _upsample_like(hx4d, hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4dup, hx3), 1))
        hx3dup = _upsample_like(hx3d, hx2)

        hx2d = self.rebnconv2d(torch.cat((hx3dup, hx2), 1))
        hx2dup = _upsample_like(hx2d, hx1)

        hx1d = self.rebnconv1d(torch.cat((hx2dup, hx1), 1))

        return hx1d + hxin


class RSU6(nn.Module):
    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU6, self).__init__()

        self.rebnconvin = REBNCONV(in_ch, out_ch, dirate=1)

        self.rebnconv1 = REBNCONV(out_ch, mid_ch, dirate=1)
        self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv2 = REBNCONV(mid_ch, mid_ch, dirate=1)
        self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv3 = REBNCONV(mid_ch, mid_ch, dirate=1)
        self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv4 = REBNCONV(mid_ch, mid_ch, dirate=1)
        self.pool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv5 = REBNCONV(mid_ch, mid_ch, dirate=1)

        self.rebnconv6 = REBNCONV(mid_ch, mid_ch, dirate=2)

        self.rebnconv5d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv4d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv3d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv2d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv1d = REBNCONV(mid_ch * 2, out_ch, dirate=1)

    def forward(self, x):
        hx = x

        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx = self.pool1(hx1)

        hx2 = self.rebnconv2(hx)
        hx = self.pool2(hx2)

        hx3 = self.rebnconv3(hx)
        hx = self.pool3(hx3)

        hx4 = self.rebnconv4(hx)
        hx = self.pool4(hx4)

        hx5 = self.rebnconv5(hx)

        hx6 = self.rebnconv6(hx5)

        hx5d = self.rebnconv5d(torch.cat((hx6, hx5), 1))
        hx5dup = _upsample_like(hx5d, hx4)

        hx4d = self.rebnconv4d(torch.cat((hx5dup, hx4), 1))
        hx4dup = _upsample_like(hx4d, hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4dup, hx3), 1))
        hx3dup = _upsample_like(hx3d, hx2)

        hx2d = self.rebnconv2d(torch.cat((hx3dup, hx2), 1))
        hx2dup = _upsample_like(hx2d, hx1)

        hx1d = self.rebnconv1d(torch.cat((hx2dup, hx1), 1))

        return hx1d + hxin


class RSU5(nn.Module):
    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU5, self).__init__()

        self.rebnconvin = REBNCONV(in_ch, out_ch, dirate=1)

        self.rebnconv1 = REBNCONV(out_ch, mid_ch, dirate=1)
        self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv2 = REBNCONV(mid_ch, mid_ch, dirate=1)
        self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv3 = REBNCONV(mid_ch, mid_ch, dirate=1)
        self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv4 = REBNCONV(mid_ch, mid_ch, dirate=1)

        self.rebnconv5 = REBNCONV(mid_ch, mid_ch, dirate=2)

        self.rebnconv4d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv3d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv2d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv1d = REBNCONV(mid_ch * 2, out_ch, dirate=1)

    def forward(self, x):
        hx = x

        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx = self.pool1(hx1)

        hx2 = self.rebnconv2(hx)
        hx = self.pool2(hx2)

        hx3 = self.rebnconv3(hx)
        hx = self.pool3(hx3)

        hx4 = self.rebnconv4(hx)

        hx5 = self.rebnconv5(hx4)

        hx4d = self.rebnconv4d(torch.cat((hx5, hx4), 1))
        hx4dup = _upsample_like(hx4d, hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4dup, hx3), 1))
        hx3dup = _upsample_like(hx3d, hx2)

        hx2d = self.rebnconv2d(torch.cat((hx3dup, hx2), 1))
        hx2dup = _upsample_like(hx2d, hx1)

        hx1d = self.rebnconv1d(torch.cat((hx2dup, hx1), 1))

        return hx1d + hxin


class RSU4(nn.Module):
    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU4, self).__init__()

        self.rebnconvin = REBNCONV(in_ch, out_ch, dirate=1)

        self.rebnconv1 = REBNCONV(out_ch, mid_ch, dirate=1)
        self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv2 = REBNCONV(mid_ch, mid_ch, dirate=1)
        self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.rebnconv3 = REBNCONV(mid_ch, mid_ch, dirate=1)

        self.rebnconv4 = REBNCONV(mid_ch, mid_ch, dirate=2)

        self.rebnconv3d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv2d = REBNCONV(mid_ch * 2, mid_ch, dirate=1)
        self.rebnconv1d = REBNCONV(mid_ch * 2, out_ch, dirate=1)

    def forward(self, x):
        hx = x

        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx = self.pool1(hx1)

        hx2 = self.rebnconv2(hx)
        hx = self.pool2(hx2)

        hx3 = self.rebnconv3(hx)

        hx4 = self.rebnconv4(hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4, hx3), 1))
        hx3dup = _upsample_like(hx3d, hx2)

        hx2d = self.rebnconv2d(torch.cat((hx3dup, hx2), 1))
        hx2dup = _upsample_like(hx2d, hx1)

        hx1d = self.rebnconv1d(torch.cat((hx2dup, hx1), 1))

        return hx1d + hxin


class RSU4F(nn.Module):
    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU4F, self).__init__()

        self.rebnconvin = REBNCONV(in_ch, out_ch, dirate=1)

        self.rebnconv1 = REBNCONV(out_ch, mid_ch, dirate=1)
        self.rebnconv2 = REBNCONV(mid_ch, mid_ch, dirate=2)
        self.rebnconv3 = REBNCONV(mid_ch, mid_ch, dirate=4)

        self.rebnconv4 = REBNCONV(mid_ch, mid_ch, dirate=8)

        self.rebnconv3d = REBNCONV(mid_ch * 2, mid_ch, dirate=4)
        self.rebnconv2d = REBNCONV(mid_ch * 2, mid_ch, dirate=2)
        self.rebnconv1d = REBNCONV(mid_ch * 2, out_ch, dirate=1)

    def forward(self, x):
        hx = x

        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx2 = self.rebnconv2(hx1)
        hx3 = self.rebnconv3(hx2)

        hx4 = self.rebnconv4(hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4, hx3), 1))
        hx2d = self.rebnconv2d(torch.cat((hx3d, hx2), 1))
        hx1d = self.rebnconv1d(torch.cat((hx2d, hx1), 1))

        return hx1d + hxin

class U2NET(nn.Module):
    def __init__(self, in_ch=3, out_ch=1):
        super(U2NET, self).__init__()

        self.stage1 = RSU7(in_ch, 32, 64)
        self.pool12 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.stage2 = RSU6(64, 32, 128)
        self.pool23 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.stage3 = RSU5(128, 64, 256)
        self.pool34 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.stage4 = RSU4(256, 128, 512)
        self.pool45 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.stage5 = RSU4F(512, 256, 512)
        self.pool56 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.stage6 = RSU4F(512, 256, 512)

        self.stage5d = RSU4F(1024, 256, 512)
        self.stage4d = RSU4(1024, 128, 256)
        self.stage3d = RSU5(512, 64, 128)
        self.stage2d = RSU6(256, 32, 64)
        self.stage1d = RSU7(128, 16, 64)

        self.side1 = nn.Conv2d(64, out_ch, 3, padding=1)
        self.side2 = nn.Conv2d(64, out_ch, 3, padding=1)
        self.side3 = nn.Conv2d(128, out_ch, 3, padding=1)
        self.side4 = nn.Conv2d(256, out_ch, 3, padding=1)
        self.side5 = nn.Conv2d(512, out_ch, 3, padding=1)
        self.side6 = nn.Conv2d(512, out_ch, 3, padding=1)

        self.outconv = nn.Conv2d(6 * out_ch, out_ch, 1)

    def forward(self, x):
        hx = x

        hx1 = self.stage1(hx)
        hx = self.pool12(hx1)

        hx2 = self.stage2(hx)
        hx = self.pool23(hx2)

        hx3 = self.stage3(hx)
        hx = self.pool34(hx3)

        hx4 = self.stage4(hx)
        hx = self.pool45(hx4)

        hx5 = self.stage5(hx)
        hx = self.pool56(hx5)

        hx6 = self.stage6(hx)
        hx6up = _upsample_like(hx6, hx5)

        hx5d = self.stage5d(torch.cat((hx6up, hx5), 1))
        hx5dup = _upsample_like(hx5d, hx4)

        hx4d = self.stage4d(torch.cat((hx5dup, hx4), 1))
        hx4dup = _upsample_like(hx4d, hx3)

        hx3d = self.stage3d(torch.cat((hx4dup, hx3), 1))
        hx3dup = _upsample_like(hx3d, hx2)

        hx2d = self.stage2d(torch.cat((hx3dup, hx2), 1))
        hx2dup = _upsample_like(hx2d, hx1)

        hx1d = self.stage1d(torch.cat((hx2dup, hx1), 1))

        d1 = self.side1(hx1d)

        d2 = self.side2(hx2d)
        d2 = _upsample_like(d2, d1)

        d3 = self.side3(hx3d)
        d3 = _upsample_like(d3, d1)

        d4 = self.side4(hx4d)
        d4 = _upsample_like(d4, d1)

        d5 = self.side5(hx5d)
        d5 = _upsample_like(d5, d1)

        d6 = self.side6(hx6)
        d6 = _upsample_like(d6, d1)

        d0 = self.outconv(torch.cat((d1, d2, d3, d4, d5, d6), 1))

        return d0, d1, d2, d3, d4, d5, d6


In [ ]:
#@title Install gradio
!pip install gradio

In [ ]:
import gradio as gr

content_image = './examples/dress.jpg'
style_image = './examples/style.jpg'
generated_image = './examples/generated.png'
output_image = './examples/output.png'
saliency_image = './examples/saliency_map.jpg'
mask_image = './examples/mask.jpg'

content_image = gr.inputs.Image(label="Upload Cloth Image", type="filepath")
style_image = gr.inputs.Image(label="Upload Style Image",type="filepath")

lr_slider = gr.inputs.Slider(0.001, 0.1, label="Adjust Learning Rate" , default=0.01, step=0.001)
epoch_slider = gr.inputs.Slider(100, 1000, label="Number of epochs" , default=100,step=50,)


def perform_design_transfer(content_image, style_image, lr, epochs):
    if ENABLE_STYLE_TRANSFER:
        style_transfer_pipeline(load_vgg_model(), content_image, style_image, int(epochs), lr, output_image=generated_image)

    mask = segmentation_pipeline(load_u2net_model(), content_image, output_image=mask_image)
    saliency_map = saliency_map_pipeline(
        load_u2net_model(out_ch=1, checkpoint=U2NET_SALIENCY_MAP_CHECKPOINT_FILE, ordered_dict=False),
        content_image,
        output_image=saliency_image
    )
    output = merge_images_pipeline(mask, saliency_map, content_image, generated_image, output_image=output_image)
    return output

merged_image =  gr.outputs.Image(label = "Output",type="filepath").style(height=500)

example1 = ['./examples/example-1/dress.jpg', './examples/example-1/style.jpg']
example2 = ['./examples/example-2/dress.jpg', './examples/example-2/style.jpg']

app_interface = gr.Interface(fn=perform_design_transfer,
                             inputs=[content_image,
                                     style_image,
                                     lr_slider,
                                     epoch_slider,
                                     ],
                             outputs=merged_image,
                             examples=[example1, example2],
                             title="Design2Style",
                             description="Interactive Clothing Image Generation combines neural style transfer, U2-Net segmentation, and saliency mapping to transform and personalize clothing articles."
                             )

app_interface.queue().launch(debug=True,share=True)


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:259: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:262: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:93: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:43: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/d

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://889cdf3423a3ba104b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Epoch[0] Total loss = 9824051.0
Epoch[1] Total loss = 9431073.0
Epoch[2] Total loss = 8906381.0
Epoch[3] Total loss = 8317710.5
Epoch[4] Total loss = 7717464.5
Epoch[5] Total loss = 7148005.5
Epoch[6] Total loss = 6644780.5
Epoch[7] Total loss = 6209053.5
Epoch[8] Total loss = 5815017.5
Epoch[9] Total loss = 5447209.5
Epoch[10] Total loss = 5112764.5
Epoch[11] Total loss = 4824786.5
Epoch[12] Total loss = 4585852.0
Epoch[13] Total loss = 4388284.0
Epoch[14] Total loss = 4221011.5
Epoch[15] Total loss = 4074113.0
Epoch[16] Total loss = 3941601.0
Epoch[17] Total loss = 3820402.75
Epoch[18] Total loss = 3709091.25
Epoch[19] Total loss = 3606762.75
Epoch[20] Total loss = 3512101.75
Epoch[21] Total loss = 3423551.5
Epoch[22] Total loss = 3339463.25
Epoch[23] Total loss = 3258582.75
Epoch[24] Total loss = 3180203.5
Epoch[25] Total loss = 3104051.0
Epoch[26] Total loss = 3029819.25
Epoch[27] Total loss = 2957291.25
Epoch[28] Total loss = 2886225.0
Epoch[29] Total loss = 2816373.5
Epoch[30] To